In [27]:
import QuantLib as ql
import pandas as pd

In [28]:
#today = ql.Date().todaysDate()
today = ql.Date(12,4,2017)
ql.Settings.instance().evaluationDate = today

#### Cashflows

In [29]:
cf = pd.read_excel('cashflows.xlsx')
cf['Date'] = cf['Date'].apply(lambda x: ql.Date(x.strftime('%Y-%m-%d'), '%Y-%m-%d'))
cf['simple_cf'] = cf.apply(lambda x: ql.SimpleCashFlow(x['RealCF'], x['Date']), axis=1)
leg = ql.Leg(list(cf['simple_cf']))
cf

,Date,RealCF,simple_cf
0,"June 21st, 2017",3.988382,<QuantLib.QuantLib.SimpleCashFlow; proxy of <S...
1,"December 21st, 2017",4.010959,<QuantLib.QuantLib.SimpleCashFlow; proxy of <S...
2,"June 21st, 2018",3.989041,<QuantLib.QuantLib.SimpleCashFlow; proxy of <S...
3,"December 21st, 2018",104.010959,<QuantLib.QuantLib.SimpleCashFlow; proxy of <S...


#### Curve

In [30]:
my_curve = pd.read_excel('yield_curve.xlsx')
tenor = list(my_curve['Tenor'].apply(lambda x: ql.Date(x.strftime('%Y-%m-%d'), '%Y-%m-%d')))
zeros = list(my_curve['Yield'])
zero_curve = ql.ZeroCurve(tenor, zeros, ql.Actual365Fixed(), ql.TARGET(), ql.Linear(), ql.Semiannual)
zero_curve_handle = ql.YieldTermStructureHandle(zero_curve)
#my_curve

#### NPV

In [31]:
npv = ql.CashFlows.npv(leg, zero_curve_handle, True, today)
npv

103.1052961341164

#### YTM

In [32]:
ytm = ql.CashFlows.yieldRate(leg, npv, ql.ActualActual(), ql.Compounded, ql.Annual, True, today)
ytm

0.07715053982026873

#### Modified Duration

In [33]:
modified_duration = ql.CashFlows.duration(leg, ytm, ql.ActualActual(), ql.Compounded, ql.Annual, ql.Duration.Modified, True, today)
modified_duration

1.467941552037983

#### Convexity

In [34]:
convexity = ql.CashFlows.convexity(leg, ytm, ql.ActualActual(), ql.Compounded, ql.Annual, True, today)
convexity

3.620454794239221

#### PV01

In [35]:
PV01 = ql.CashFlows.bps(leg, ytm, ql.ActualActual(), ql.Compounded, ql.Annual, True, today)
PV01

0.0

#### RPP

In [36]:
RPP = -ql.CashFlows.basisPointValue(leg, ytm, ql.ActualActual(), ql.Compounded, ql.Annual, True, today)*10000
RPP

151.35236178996672